In [17]:
import pandas as pd
import numpy as np
import os
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import json

In [18]:
csv_path = r'F:\dataset\merged csv\dataset_balance.csv'
df = pd.read_csv(csv_path)

dict_2classes = {'DDoS-SlowLoris': 'Attack', 'BenignTraffic': 'Benign'}
df['label'] = df['label'].map(dict_2classes)
df = df.dropna(subset=['label'])

In [19]:
X_columns = [
    'flow_duration', 'Header_Length', 'Protocol Type', 'Duration',
    'Rate', 'Srate', 'Drate', 'fin_flag_number', 'syn_flag_number',
    'rst_flag_number', 'psh_flag_number', 'ack_flag_number',
    'ece_flag_number', 'cwr_flag_number', 'ack_count',
    'syn_count', 'fin_count', 'urg_count', 'rst_count', 
    'HTTP', 'HTTPS', 'DNS', 'Telnet', 'SMTP', 'SSH', 'IRC', 'TCP',
    'UDP', 'DHCP', 'ARP', 'ICMP', 'IPv', 'LLC', 'Tot sum', 'Min',
    'Max', 'AVG', 'Std', 'Tot size', 'IAT', 'Number', 'Magnitue',
    'Radius', 'Covariance', 'Variance', 'Weight'
]
X = df[X_columns]
y = df['label']

In [20]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

In [22]:
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear'],  
    'class_weight': ['balanced']
}

grid_search = GridSearchCV(
    estimator=LogisticRegression(n_jobs=-1, max_iter=1000),
    param_grid=param_grid,
    cv=5,
    scoring='f1_macro',
    verbose=0
)

start_train_time = time.time()
grid_search.fit(X_train, y_train)
end_train_time = time.time()

best_model = grid_search.best_estimator_

In [23]:
start_pred_time = time.time()
y_pred = best_model.predict(X_test)
end_pred_time = time.time()
training_duration = end_train_time - start_train_time
prediction_duration = end_pred_time - start_pred_time

In [24]:
print("Calculating per-case prediction time...")
per_case_times = []
for i in tqdm(range(len(X_test))):
    x_single = X_test[i].reshape(1, -1)
    start_case = time.time()
    _ = best_model.predict(x_single)
    end_case = time.time()
    elapsed_ms = (end_case - start_case) * 1000
    per_case_times.append(elapsed_ms)

Calculating per-case prediction time...


100%|██████████| 9371/9371 [00:00<00:00, 16717.47it/s]


In [25]:
per_case_df = pd.DataFrame({'CaseIndex': range(len(per_case_times)), 'PredictionTime_ms': per_case_times})
per_case_df.to_csv("per_case_prediction_time_LR_optimal.csv", index=False)
print("Per-case prediction time saved to per_case_prediction_time_LR_optimal.csv.")

Per-case prediction time saved to per_case_prediction_time_LR_optimal.csv.


In [26]:
average_case_time_ms = np.mean(per_case_times)
print(f"Average Prediction Time per Case: {average_case_time_ms:.4f} ms")

Average Prediction Time per Case: 0.0586 ms


In [27]:
acc = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

print("##### Optimized Logistic Regression (2-class) #####")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred, average='macro'))
print("Precision:", precision_score(y_test, y_pred, average='macro'))
print("F1 Score:", f1_score(y_test, y_pred, average='macro'))
print(f"Training Time: {training_duration:.4f} seconds")
print(f"Prediction Time: {prediction_duration:.4f} seconds")

##### Optimized Logistic Regression (2-class) #####
Accuracy: 0.8281933625013339
Recall: 0.8281959568933279
Precision: 0.8289715020723533
F1 Score: 0.8280924895588302
Training Time: 30.2006 seconds
Prediction Time: 0.0010 seconds


In [28]:
result = {
    "Model": "LR_optimal",
    "Accuracy": round(float(acc), 6),
    "Recall": round(float(recall), 6),
    "Precision": round(float(precision), 6),
    "F1 Score": round(float(f1), 6),
    "Training Time (s)": round(float(training_duration), 4),
    "Prediction Time (s)": round(float(prediction_duration), 4),
    "Avg Case Predict Time (ms)": round(float(average_case_time_ms), 4)
}

with open("result_LR_optimal.json", "w") as f:
    json.dump(result, f, indent=4)

print("Model evaluation results have been saved to result_LR_optimal.json.")

Model evaluation results have been saved to result_LR_optimal.json.


In [40]:
import pandas as pd
import numpy as np
import os
import time
import warnings
import random
import json
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.feature_selection import SelectKBest, f_classif
from tqdm import tqdm

# ===== 配置 =====
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
warnings.filterwarnings('ignore')

# ===== 数据加载 =====
csv_path = r'F:\dataset\merged csv\dataset_balance.csv'
df = pd.read_csv(csv_path)

# 标签映射
df['label'] = df['label'].map({
    'DDoS-SlowLoris': 'Attack',
    'BenignTraffic': 'Benign'
})
df = df.dropna(subset=['label'])
df['label'] = df['label'].map({'Benign': 0, 'Attack': 1})

# 特征列定义
X_columns = [  # 包含 flow_duration
    'flow_duration', 'Header_Length', 'Protocol Type', 'Duration',
    'Rate', 'Srate', 'Drate', 'fin_flag_number', 'syn_flag_number',
    'rst_flag_number', 'psh_flag_number', 'ack_flag_number',
    'ece_flag_number', 'cwr_flag_number', 'ack_count', 'syn_count',
    'fin_count', 'urg_count', 'rst_count', 'HTTP', 'HTTPS', 'DNS',
    'Telnet', 'SMTP', 'SSH', 'IRC', 'TCP', 'UDP', 'DHCP', 'ARP',
    'ICMP', 'IPv', 'LLC', 'Tot sum', 'Min', 'Max', 'AVG', 'Std',
    'Tot size', 'IAT', 'Number', 'Magnitue', 'Radius', 'Covariance',
    'Variance', 'Weight'
]

X = df[X_columns].astype('float32')
y = df['label'].astype('int32')

# 特征标准化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 特征选择（k=30）
k_best = 30
selector = SelectKBest(score_func=f_classif, k=k_best)
X_selected = selector.fit_transform(X_scaled, y)
selected_features = np.array(X_columns)[selector.get_support()]

# 保存 flow_duration（未经缩放的原始值）
flow_duration = df['flow_duration'].reset_index(drop=True)

# 训练/测试集划分
X_train, X_test, y_train, y_test, flow_train, flow_test = train_test_split(
    X_selected, y, flow_duration,
    test_size=0.2, random_state=SEED, stratify=y, shuffle=True
)

# 模型训练
model = LogisticRegression(
    solver='liblinear',
    penalty='l2',
    class_weight='balanced',
    max_iter=2000,
    random_state=SEED
)

start_train_time = time.time()
model.fit(X_train, y_train)
end_train_time = time.time()
training_duration = end_train_time - start_train_time

# 推理整体时间
start_pred_time = time.time()
y_pred = model.predict(X_test)
end_pred_time = time.time()
prediction_duration = end_pred_time - start_pred_time

# ==== 新增：每个 case 的推理时间 ====
per_case_times = []
for i in tqdm(range(len(X_test))):
    x_single = X_test[i].reshape(1, -1)
    start_case = time.time()
    _ = model.predict(x_single)
    end_case = time.time()
    elapsed_ms = (end_case - start_case) * 1000  # ms
    per_case_times.append(elapsed_ms)

# 保存每条样本的预测时间与 flow_duration 对比
comparison_df = pd.DataFrame({
    'PredictionTime_ms': per_case_times,
    'FlowDuration': flow_test.reset_index(drop=True)
})
comparison_df.to_csv('flow_vs_prediction_time.csv', index=False)

# 平均值对比输出
avg_pred_time = np.mean(per_case_times)
avg_flow_duration = np.mean(comparison_df['FlowDuration'])

print(f"\n📊 Avg Prediction Time per Case: {avg_pred_time:.4f} ms")
print(f"📊 Avg Flow Duration per Case: {avg_flow_duration:.2f} ms")

# ==== 性能评估 ====
acc = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# 输出评估结果
print("\n##### Optimized Logistic Regression with Timing #####")
print("Accuracy:", acc)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print(f"Training Time: {training_duration:.4f} seconds")
print(f"Prediction Time (total): {prediction_duration:.4f} seconds")

# 最终结果保存
result = {
    "Model": "LR_k30_L2_balanced",
    "Selected Features": selected_features.tolist(),
    "Accuracy": float(acc),
    "Precision": float(precision),
    "Recall": float(recall),
    "F1": float(f1),
    "Train Time (s)": float(training_duration),
    "Predict Time (s)": float(prediction_duration),
    "Avg Prediction Time per Case (ms)": float(avg_pred_time),
    "Avg Flow Duration per Case (ms)": float(avg_flow_duration)
}

with open("result_LR_k30_with_time.json", "w") as f:
    json.dump(result, f, indent=4)

print("✅ Result and timing comparison saved.")


100%|██████████| 9371/9371 [00:00<00:00, 15320.21it/s]


📊 Avg Prediction Time per Case: 0.0623 ms
📊 Avg Flow Duration per Case: 34.37 ms

##### Optimized Logistic Regression with Timing #####
Accuracy: 0.8327819869811119
Precision: 0.859713890170743
Recall: 0.7953041622198506
F1 Score: 0.826255682448165
Training Time: 0.3314 seconds
Prediction Time (total): 0.0010 seconds
✅ Result and timing comparison saved.
